In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
save_path = '../demo-out/'
exp = 'all_cd14_monocytes'
mod = 'rna_'
file = 'L10_D5_'

In [4]:
from scphere.util.util import read_mtx
import numpy as np
from scphere.model.vae import SCPHERE
from scphere.util.trainer import Trainer

import anndata as ad

/home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/users/nlaa

In [5]:
all_rna_path = '/research/groups/sysgen/PROJECTS/scHEMAP/janne_work/Easigenomics/Python/Donor_withLabels/L10_D5_data_lt.h5ad'
all_rna = ad.read_h5ad(all_rna_path)
all_rna.var_names_make_unique()
all_rna

AnnData object with n_obs × n_vars = 6733 × 32738
    obs: 'nCount_RNA', 'nFeature_RNA', 'bc', 'propmt', 'rna_size', 'ngene', 'prot_size', 'predicted.id', 'hashMax', 'adtMax', 'nHashPos', 'HTO_margin', 'prediction.score.CD14..monocyte.1', 'prediction.score.CD14..monocyte.2', 'prediction.score.CD16..monocyte', 'prediction.score.5_preB_G2MS', 'prediction.score.HSC', 'prediction.score.NK', 'prediction.score.CD4..T.1', 'prediction.score.NK.T.1', 'prediction.score.NK.T.2', 'prediction.score.CD8..T', 'prediction.score.CD4..T.3', 'prediction.score.6_preB.II_G1', 'prediction.score.6_preB_G1', 'prediction.score.7_immatureB', 'prediction.score.Late.erythoid.precursor.2', 'prediction.score.Early.erythoid.precursor', 'prediction.score.4_proB_G1', 'prediction.score.2_earlyLymphoid', 'prediction.score.Plasma.B.cell', 'prediction.score.3_proB_G2MS', 'prediction.score.Megakaryocyte', 'prediction.score.max'
    var: 'name'

In [6]:
# all_prot_path = '/research/groups/sysgen/PROJECTS/scHEMAP/janne_work/Easigenomics/Python/Donor_withLabels/L10_D5_dsbprot_data_lt.h5ad'
# all_prot = ad.read_h5ad(all_prot_path)
# all_prot.var_names_make_unique()
# all_prot

In [7]:
all_rna.obs['predicted.id'].unique().describe()

,counts,freqs
categories,,
5_preB_G2MS,1,0.071429
7_immatureB,1,0.071429
CD14+ monocyte 2,1,0.071429
NK T 1,1,0.071429
HSC,1,0.071429
6_preB_G1,1,0.071429
CD4+ T 1,1,0.071429
CD4+ T 3,1,0.071429
NK,1,0.071429


In [8]:
cell_types = ['CD14+ monocyte 2', 'CD14+ monocyte 1', 'CD16+ monocyte']
data = all_rna[all_rna.obs['predicted.id'].isin(cell_types)]
data

View of AnnData object with n_obs × n_vars = 130 × 32738
    obs: 'nCount_RNA', 'nFeature_RNA', 'bc', 'propmt', 'rna_size', 'ngene', 'prot_size', 'predicted.id', 'hashMax', 'adtMax', 'nHashPos', 'HTO_margin', 'prediction.score.CD14..monocyte.1', 'prediction.score.CD14..monocyte.2', 'prediction.score.CD16..monocyte', 'prediction.score.5_preB_G2MS', 'prediction.score.HSC', 'prediction.score.NK', 'prediction.score.CD4..T.1', 'prediction.score.NK.T.1', 'prediction.score.NK.T.2', 'prediction.score.CD8..T', 'prediction.score.CD4..T.3', 'prediction.score.6_preB.II_G1', 'prediction.score.6_preB_G1', 'prediction.score.7_immatureB', 'prediction.score.Late.erythoid.precursor.2', 'prediction.score.Early.erythoid.precursor', 'prediction.score.4_proB_G1', 'prediction.score.2_earlyLymphoid', 'prediction.score.Plasma.B.cell', 'prediction.score.3_proB_G2MS', 'prediction.score.Megakaryocyte', 'prediction.score.max'
    var: 'name'

In [9]:
cell_types = data.obs['predicted.id'].to_numpy()
np.savetxt(save_path + exp + mod + file +
           'cite_cell_types.tsv',
           cell_types, fmt='%s')


In [9]:
x = data.X.toarray().astype(dtype='int')

In [10]:
batch = np.zeros(x.shape[0]) * -1

In [11]:
model = SCPHERE(n_gene=x.shape[1], n_batch=0, batch_invariant=False,
                z_dim=2, latent_dist='vmf',
                observation_dist='nb', seed=0)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2022-03-01 14:51:09,814: WARNING: 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-03-01 14:51:09,830: WARNING: From /home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2022-03-01 14:51:10,054: WARNING: From /home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:2403: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


2022-03-01 14:51:10,676: WARNING: From /home/users/nlaabid/miniconda3/envs/scphere2/lib/python3.6/site-packages/scPhere-0.1.0-py3.6.egg/scphere/ops/ive.py:23: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [12]:
trainer = Trainer(model=model, x=x, batch_id=batch, max_epoch=250, mb_size=128, learning_rate=0.001)
trainer.train()

0 / 250 {'Log-likelihood': -16280.863, 'ELBO': -16281.243, 'KL': 0.37976545}
50 / 250 {'Log-likelihood': -7778.6377, 'ELBO': -7781.265, 'KL': 2.6273143}
100 / 250 {'Log-likelihood': -6394.6406, 'ELBO': -6397.585, 'KL': 2.9444675}


In [ ]:
model_name = save_path + exp + mod + file + 'cd14_mono_eryth_model_250epoch'
model.save_sess(model_name)

In [ ]:
type(batch)

In [ ]:
z_mean = model.encode(x, batch)
np.savetxt(save_path + exp + mod + file +
           'cd14_mono_eryth_latent_250epoch.tsv',
           z_mean)


In [ ]:
ll = model.get_log_likelihood(x, batch)
np.savetxt(save_path + exp + mod + file +
           'cd14_mono_eryth_ll_250epoch.tsv',
           z_mean)

In [ ]:
# why is one float and one int?
# read study for data collection of scphere data